In [ ]:
# default_exp fit

# neos.fit

> Module containing functions to perform maximum likelihod fits in a differentiable way.

The `fax` module is leveraged to calculate gradients of maximum likelihood fits through the 'two-phase' method, which is a technique for automatic differentiation of functions with an attractive fixed point, i.e. they satisfy $f(x) - x = 0$. This is the case for a minimization routine, where the minimization occurs from the minimum: `minimize(f, x_init) = x_min` --> `minimize(f, x_init=x_min) = x_min`. You can imagine that the initial iterations won't give any useful information in terms of the gradient of the minimum; this approach leverages this fact by performing the minimization a second time in the neighborhood of the fixed point, and keeping track of gradients there, which avoids the unnecessary unrolling of many loops for the early iteration.

To read more about this method, see section 2.3 of [this paper](https://www.researchgate.net/profile/Ala_Taftaf2/publication/323176030_ADJOINTS_OF_FIXED-POINT_ITERATIONS/links/5a8465644585159152b7fe00/ADJOINTS-OF-FIXED-POINT-ITERATIONS.pdf) that looks at some methods to approach the fixed-point differation problem within automatic differentiation.

The fits themselves are done by gradient descent with Adam.

In [ ]:
#export
import jax
import jax.numpy as jnp

from fax.implicit import twophase
import jax.experimental.optimizers as optimizers

from neos.transforms import to_bounded_vec, to_inf_vec, to_bounded, to_inf

In [ ]:
#export 
def global_fit(
    model_constructor,
    pdf_transform=False,
    default_rtol=1e-10,
    default_atol=1e-10,
    default_max_iter=int(1e7),
    learning_rate=1e-6,
):
    """Wraps a series of functions that perform maximum likelihood fitting in
    the `two_phase_solver` method found in the `fax` python module. This allows
    for the calculation of gradients of the best-fit parameters with respect to
    upstream parameters that control the underlying model, i.e. the event
    yields (which are then parameterized by weights or similar).

    Args:
        model_constructor: Function that takes in the parameters of the observable,
        and returns a model object (and background-only parameters)
    Returns:
        global_fitter: Callable function that performs global fits.
        Differentiable :)
    """

    adam_init, adam_update, adam_get_params = optimizers.adam(learning_rate)

    def make_model(model_pars):
        m, bonlypars = model_constructor(model_pars)

        bounds = m.config.suggested_bounds()

        exp_bonly_data = m.expected_data(bonlypars, include_auxdata=True)

        def expected_logpdf(pars):  # maps pars to bounded space if pdf_transform = True

            return (
                m.logpdf(to_bounded_vec(pars, bounds), exp_bonly_data)
                if pdf_transform
                else m.logpdf(pars, exp_bonly_data)
            )

        def global_fit_objective(pars):  # NLL
            return -expected_logpdf(pars)[0]

        return global_fit_objective

    def global_bestfit_minimized(hyper_param):
        nll = make_model(hyper_param)

        def bestfit_via_grad_descent(i, param):  # gradient descent
            g = jax.grad(nll)(param)
            # param = param - g * learning_rate
            param = adam_get_params(adam_update(i, g, adam_init(param)))
            return param

        return bestfit_via_grad_descent

    global_solve = twophase.two_phase_solver(
        param_func=global_bestfit_minimized,
        default_rtol=default_rtol,
        default_atol=default_atol,
        default_max_iter=default_max_iter,
    )

    def global_fitter(init, hyper_pars):
        solve = global_solve(init, hyper_pars)
        return solve.value

    return global_fitter

In [ ]:
#export
def constrained_fit(
    model_constructor,
    pdf_transform=False,
    default_rtol=1e-10,
    default_atol=1e-10,
    default_max_iter=int(1e7),
    learning_rate=1e-6,
):
    """Wraps a series of functions that perform maximum likelihood fitting in
    the `two_phase_solver` method found in the `fax` python module. This allows
    for the calculation of gradients of the best-fit parameters with respect to
    upstream parameters that control the underlying model, i.e. the event
    yields (which are then parameterized by weights or similar).

    Args:
        model_constructor: Function that takes in the parameters of the observable,
        and returns a model object (and background-only parameters)
    Returns:
        constrained_fitter: Callable function that performs constrained fits.
        Differentiable :)
    """

    adam_init, adam_update, adam_get_params = optimizers.adam(learning_rate)

    def make_model(hyper_pars):

        model_pars, constrained_mu = hyper_pars
        m, bonlypars = model_constructor(model_pars)

        bounds = m.config.suggested_bounds()
        constrained_mu = (
            to_inf(constrained_mu, bounds[0]) if pdf_transform else constrained_mu
        )

        exp_bonly_data = m.expected_data(bonlypars, include_auxdata=True)

        def expected_logpdf(pars):  # maps pars to bounded space if pdf_transform = True

            return (
                m.logpdf(to_bounded_vec(pars, bounds), exp_bonly_data)
                if pdf_transform
                else m.logpdf(pars, exp_bonly_data)
            )

        def constrained_fit_objective(nuis_par):  # NLL
            pars = jnp.concatenate([jnp.asarray([constrained_mu]), nuis_par])
            return -expected_logpdf(pars)[0]

        return constrained_mu, constrained_fit_objective

    def constrained_bestfit_minimized(hyper_pars):
        mu, cnll = make_model(hyper_pars)

        def bestfit_via_grad_descent(i, param):  # gradient descent
            _, np = param[0], param[1:]
            g = jax.grad(cnll)(np)
            np = adam_get_params(adam_update(i, g, adam_init(np)))
            param = jnp.concatenate([jnp.asarray([mu]), np])
            return param

        return bestfit_via_grad_descent

    constrained_solver = twophase.two_phase_solver(
        param_func=constrained_bestfit_minimized,
        default_rtol=default_rtol,
        default_atol=default_atol,
        default_max_iter=default_max_iter,
    )

    def constrained_fitter(init, hyper_pars):
        solve = constrained_solver(init, hyper_pars)
        return solve.value

    return constrained_fitter

### Usage:

In [ ]:
import jax
import jax.numpy as jnp
from jax.random import PRNGKey
from jax.experimental import stax

import neos
from neos.makers import hists_from_nn, histosys_model_from_hists
from neos.data import generate_blobs
from neos.fit import global_fit, constrained_fit

# data generator
gen_data = generate_blobs(rng=PRNGKey(1),blobs=4)

# nn
init_random_params, predict = stax.serial(
    stax.Dense(1024),
    stax.Relu,
    stax.Dense(1),
    stax.Sigmoid
)

# instantiate model maker
hist_maker = hists_from_nn(gen_data, predict, method='kde')
model_maker = histosys_model_from_hists(hist_maker)

# grab parameters for nn and kde
_, network = init_random_params(jax.random.PRNGKey(13), (-1, 2))
hyperpars = dict(bandwidth=0.5, bins=jnp.linspace(0,1,3))

# fitting time!
g_fitter = global_fit(model_maker)
c_fitter = constrained_fit(model_maker)

# make model
m, bonlypars = model_maker([network, hyperpars])

initval = jnp.asarray([1.0, 1.0])

# the constrained fit
print(f'Constrained param fit (mu=1): {c_fitter(initval, [[network, hyperpars], 1])}')

# global fit
print(f'Global param fit: {g_fitter(initval, [network, hyperpars])}')

Constrained param fit (mu=1): [1.         0.02099289]
Global param fit: [-1.44533048e-06  1.29791259e-06]
